<a href="https://colab.research.google.com/github/springboardmentor891v/Email_Assistant_Using_LangGraph/blob/manoj_preetham/Email_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client


In [31]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [ ]:
# 1. SETUP GMAIL AUTHENTICATION
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credential.json',
                SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'
            )

            auth_url, _ = flow.authorization_url(
                prompt='consent'
            )

            print("\n🔗 OPEN THIS URL IN YOUR BROWSER:\n")
            print(auth_url)

            code = input("\n📌 Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()


🔗 OPEN THIS URL IN YOUR BROWSER:

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=915502763627-c2ilmm6t0n042r0lbjg7mjr0f0mmqehn.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=vY3KKjajbADFw0NBXloebmYwUBTIcU&prompt=consent&access_type=offline

📌 Paste the authorization code here: 4/1ASc3gC0h2okDoeZ46LvVMJOmJREuDRFvyWjEIN24o8aixoIPe2LSQCzV4mg


In [33]:
# =========================
# OPENAI SETUP
# =========================
api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in Colab secrets")

client = OpenAI(api_key=api_key)

def llm_call(prompt: str):
    r = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return r.choices[0].message.content.strip()


In [34]:
import os
from openai import OpenAI

# Google Colab specific import for secrets
from google.colab import userdata

# Read API key from Colab secrets
api_key = userdata.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in Colab secrets. Please ensure it's set in the Colab 'Secrets' tab.")

# Create OpenAI client
client = OpenAI(api_key=api_key)

try:
    # List models available for this key
    models = client.models.list()

    print("Models available for this key:")
    for model in models.data:
        print(model.id)

except Exception as e:
    print("Error:", e)


Models available for this key:
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano

In [35]:
# =========================
# READ EMAIL
# =========================
def extract_email_data(service, msg_id):
    message = service.users().messages().get(
        userId='me',
        id=msg_id,
        format='full'
    ).execute()

    headers = message['payload']['headers']
    subject = sender = date = ""

    for h in headers:
        if h['name'] == 'Subject':
            subject = h['value']
        elif h['name'] == 'From':
            sender = h['value']
        elif h['name'] == 'Date':
            date = h['value']

    body = ""
    payload = message['payload']

    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                data = part['body'].get('data')
                if data:
                    body = base64.urlsafe_b64decode(data).decode('utf-8')
    else:
        data = payload['body'].get('data')
        if data:
            body = base64.urlsafe_b64decode(data).decode('utf-8')

    return subject, sender, date, body
# =========================
# FETCH UNREAD
# =========================
results = service.users().messages().list(
    userId='me',
    labelIds=['UNREAD'],
    maxResults=100
).execute()

messages = results.get('messages', [])
print(f"📬 Unread emails found: {len(messages)}")
# =========================
# PREVIEW (YOUR STYLE)
# =========================
emails = []
for msg in messages:
    msg_id = msg['id']
    thread_id = msg['threadId']
    subject, sender, date, body = extract_email_data(service, msg_id)

    print("📧 SUBJECT:", subject)
    print("👤 FROM:", sender)
    print("📅 DATE:", date)
    print("📝 BODY (preview):", body[:300])
    print("-" * 60)

    emails.append((msg_id, thread_id, subject, sender, body))

📬 Unread emails found: 2
📧 SUBJECT: Regarding intership
👤 FROM: Manoj Preetham v k <vkmanojpreetham2005@gmail.com>
📅 DATE: Wed, 21 Jan 2026 23:06:05 +0530
📝 BODY (preview): Hey applicant,
 You have been shortlisted thankyou

------------------------------------------------------------
📧 SUBJECT: hello iam Tony stark died as a hero and reborn as a villlian named victor von doom
👤 FROM: Harshith Y <hharshithy@gmail.com>
📅 DATE: Wed, 21 Jan 2026 23:03:33 +0530
📝 BODY (preview): 

------------------------------------------------------------


In [38]:
def classify_email(subject, body):
    prompt = f"""
    Classify the following email into one of these categories: Emergency, Work, Promotion, Spam, Not-Useful, Personal.
    Only return the category name and nothing else.

    Email Subject: {subject}
    Email Body: {body}
    """
    category = llm_call(prompt)
    # Ensure the category is one of the expected ones, default to 'Not-Useful' if not
    valid_categories = ["Emergency", "Work", "Promotion", "Spam", "Not-Useful", "Personal"]
    if category not in valid_categories:
        # Try to infer if the LLM outputted something close, or default.
        # This simple check can be improved with more robust parsing/correction.
        for vc in valid_categories:
            if vc.lower() in category.lower():
                return vc
        return "Not-Useful" # Fallback if classification is ambiguous or invalid
    return category

# =========================
# UNREAD EMAIL SUMMARY
# =========================

# Fetch unread emails (keep your existing code if already present)
results = service.users().messages().list(
    userId='me',
    labelIds=['UNREAD'],
    maxResults=1000
).execute()

messages = results.get('messages', [])

stats = {
    "Emergency": 0,
    "Work": 0,
    "Promotion": 0,
    "Spam": 0,
    "Not-Useful": 0,
    "Personal": 0
}

# Quick pass to classify all unread and build summary
preview_cache = []  # store basic info so we don't refetch twice
for msg in messages:
    msg_id = msg['id']
    # Assuming read_email is defined elsewhere to return subject, sender, body
    subject, sender, date, body = extract_email_data(service, msg_id)
    # Fix: Pass only subject and body to classify_email as per its definition
    category = classify_email(subject, body)
    stats[category] = stats.get(category, 0) + 1
    preview_cache.append((msg_id, msg.get("threadId"), subject, sender, body, category))

print("\n📬 UNREAD SUMMARY")
print("Total:", len(messages))
print("🚨 Emergency:", stats["Emergency"])
print("💼 Work:", stats["Work"])
print("📢 Promotion:", stats["Promotion"])
print("🗑 Spam:", stats["Spam"])
print("❌ Not Useful:", stats["Not-Useful"])
print("👤 Personal:", stats["Personal"])



📬 UNREAD SUMMARY
Total: 2
🚨 Emergency: 0
💼 Work: 1
📢 Promotion: 0
🗑 Spam: 1
❌ Not Useful: 0
👤 Personal: 0


In [ ]:
# =========================
# PART B – SMART AGENT FLOW
# =========================

from collections import defaultdict

YOUR_NAME = "Manoj Preetham V K"
summary = defaultdict(int)

def classify_email(subject, body):
    prompt = f"""
Classify this email into ONE of these categories:
Emergency, Work, Promotion, Spam, Not-Useful, Personal

Email:
Subject: {subject}
Body: {body}

Return only the category name.
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content.strip()

def generate_options(subject, sender, body):
    prompt = f"""
You are an intelligent email assistant.

Email:
Subject: {subject}
From: {sender}
Body:
{body}

Generate THREE reply options.
Always sign every reply with this name: {YOUR_NAME}

1. Formal professional reply
2. Friendly casual reply
3. Very short reply

Format exactly like:

Option 1:
<text>

Option 2:
<text>

Option 3:
<text>
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4
    )
    return response.choices[0].message.content.strip()

def is_no_reply(body, sender):
    text = (body + " " + sender).lower()
    return any(x in text for x in ["do not reply", "no-reply", "noreply", "automated message"])

important_emails = []

for msg_id, thread_id, subject, sender, body in emails:
    category = classify_email(subject, body)
    summary[category] += 1

    if category in ["Emergency", "Work", "Personal"]:
        important_emails.append((msg_id, thread_id, subject, sender, body, category))

print("\n📊 UNREAD SUMMARY")
print(f"Total: {sum(summary.values())}")
print(f"🚨 Emergency: {summary['Emergency']}")
print(f"💼 Work: {summary['Work']}")
print(f"📢 Promotion: {summary['Promotion']}")
print(f"🗑 Spam: {summary['Spam']}")
print(f"❌ Not Useful: {summary['Not-Useful']}")
print(f"👤 Personal: {summary['Personal']}")

print("\n=========================")
print("IMPORTANT EMAILS FLOW")
print("=========================")

for msg_id, thread_id, subject, sender, body, category in important_emails:
    print("\n" + "="*70)
    print(f"📧 SUBJECT: {subject}")
    print(f"👤 FROM: {sender}")
    print(f"🏷 Category: {category}")
    print("📝 BODY:\n", body[:500])
    print("="*70)

    if is_no_reply(body, sender):
        print("🚫 This is a no-reply / automated email. Skipping.")
        continue

    options = generate_options(subject, sender, body)
    print("\n" + options)

    choice = input("\nChoose (1/2/3), 'edit' to write your own, or 's' to skip: ").strip().lower()

    if choice == 's':
        print("⏭️ Skipped")
        continue

    if choice == 'edit':
        reply_text = input("\n✏️ Enter your custom reply:\n\n").strip()
        if not reply_text.endswith(YOUR_NAME):
            reply_text += f"\n\n{YOUR_NAME}"
    elif choice in ['1', '2', '3']:
        parts = options.split("Option ")
        reply_text = ""
        for p in parts:
            if p.startswith(choice + ":"):
                reply_text = p.split(":", 1)[1].strip()
                break
        if not reply_text:
            print("⚠️ Could not parse the selected option. Skipped.")
            continue
    else:
        print("⚠️ Invalid choice. Skipped.")
        continue

    send_reply(
        service,
        to=sender,
        subject=subject,
        reply_text=reply_text,
        thread_id=thread_id
    )

    service.users().messages().modify(
        userId='me',
        id=msg_id,
        body={'removeLabelIds': ['UNREAD']}
    ).execute()

    print("📤 Reply sent and email marked as read")



📊 UNREAD SUMMARY
Total: 2
🚨 Emergency: 0
💼 Work: 1
📢 Promotion: 0
🗑 Spam: 1
❌ Not Useful: 0
👤 Personal: 0

IMPORTANT EMAILS FLOW

📧 SUBJECT: Regarding intership
👤 FROM: Manoj Preetham v k <vkmanojpreetham2005@gmail.com>
🏷 Category: Work
📝 BODY:
 Hey applicant,
 You have been shortlisted thankyou


Option 1:  
Dear [Recipient's Name],  
Thank you for the update regarding my internship application. I am excited to hear that I have been shortlisted. Please let me know the next steps in the process.  
Best regards,  
Manoj Preetham V K  

Option 2:  
Hey there!  
Thanks a lot for the great news! I'm really excited to be shortlisted for the internship. Looking forward to hearing more about the next steps!  
Cheers,  
Manoj Preetham V K  

Option 3:  
Thanks for the update!  
Manoj Preetham V K
